In [3]:
import numpy as np 
import matplotlib.pyplot as plt

%matplotlib inline

In [315]:
class Matrix(object):
    def __init__(self, data=None):
        self.data = np.array(data)
    
    def shape(self):
        return self.data.shape
    
    def __add__(self, other):
        if isinstance(other, Matrix):
            return Matrix(self.data + other.data)
        return Matrix(self.data + other)
    
    def __sub__(self, other):
        if isinstance(other, Matrix):
            return Matrix(self.data - other.data)
        return Matrix(self.data - other)
    
    def __mul__(self, other):
        if isinstance(other, Matrix):
            shape_1 = self.shape()
            shape_2 = other.shape()
            assert shape_1[1] == shape_2[0]
            result = np.zeros((shape_1[0], shape_2[1]), dtype=np.float)
            
            for i in range(shape_1[0]):
                for j in range(shape_2[1]):
                    result[i][j] = self.row(i).dot_product(other.col(j))
            return Matrix(result)

        return Matrix(self.data * other)
    
    def __truediv__(self, other):
        return Matrix(self.data / other)

    def T(self):
        return Matrix(self.data.T)
    
    def col(self, i):
        return Vector(self.data[:,i])
    
    def row(self, i):
        return Vector(self.data[i])
    
    def is_squared(self):
        shape = self.shape()
        return shape[0] == shape[1]
    
    def is_symmetrical(self):
        h, w = self.shape()
        for i in range(h):
            for j in range(w):
                if self.data[i][j] != self.data[j][i]:
                    return False
        return True
    
    def minor(self, row, column):
        return Matrix([np.hstack([row[:column], row[column + 1:]]) for row in np.vstack([self.data[:row], self.data[row + 1:]])])
    
    def det(self):
        N, _ = self.shape()
        if N == 1:
            return self.data[0][0]
        result = 0
        factor = 1
        for i in range(N):
            result += factor * self.data[0][i] * self.minor(0, i).det()
            factor *= -1
        return result

    def inv(self):
        N = self.shape()[0]
        cofactors = np.zeros((N, N))
        for row in range(N):
            for column in range(N):
                cofactors[row][column] = (-1) ** (row + column) * self.minor(row, column).det()
        adj = Matrix(cofactors).T()
        return adj / self.det()
    
    def diag(self):
        N = self.shape()[0]
        return self.data[np.arange(N), np.arange(N)]
    
    def sum(self):
        return self.data.sum()
    
    def min(self):
        return np.min(self.data)
    
    def max(self):
        return np.max(self.data)
    
    def __str__(self):
        s = ''
        for elem in self.data:
            s += str(list(elem)) + '\n'
        return s
    
    def __repr__(self):
        return str(self)

class Vector(Matrix):
    def __init__(self, data=None):
        if isinstance(data, Matrix):
            super().__init__(data.data.reshape((-1, 1)))
        else:
            super().__init__(data.reshape((-1, 1)))

    def __mul__(self, other):
        if isinstance(self, Vector):
            return Vector(self.data * other.data)
        return Vector(self.data * other)

    def dot_product(self, other):
        return (self * other).sum()
    
    def norm(self):
        return np.sqrt(self.dot_product(self))

class PMTask(object):
    def __init__(self, A, f, max_iterations=100, eps=1e-5, tau='gershgorin', verbose=False):
        self.A = A
        self.f = f
        self.N = A.shape()[0]
        self.max_iterations = max_iterations
        self.eps = eps
        self.verbose = verbose
        
        self.configure_data()
        
        # Configuring tau
        if tau == 'gershgorin':
            self.tau = self.gershgorin_approximation()
        else:
            self.tau = tau
        
        self.iteration = 0
        self.previous = Vector(np.ones((self.N, 1)) * 1e7)
        self.current = Vector(np.ones((self.N, 1)))
        
    def gershgorin_approximation(self):
        diag = self.A.diag()
        sum_a = self.A.sum() - diag.sum()
        minimum = diag - sum_a
        maximum = diag + sum_a
        lambda_min = minimum.min()
        lambda_max = maximum.max()
        tau = 2 / (lambda_min + lambda_max)
        if self.verbose:
            print('Gershgorin tau approximation: {} (lambd_min={}, lambd_max={})'.format(tau, lambda_min, lambda_max))
        return 2 / (lambda_min + lambda_max)
        
    def configure_data(self):
        if not self.A.is_squared():
            raise ValueError("A must be symmetrical")
        if not self.A.is_symmetrical():
            B = self.A.T() * self.A
            F = self.A.T() * self.f
            self.A, self.f = B, F
    
    def run(self):
        while self.iteration < self.max_iterations and self.current_eps() > self.eps:
            self.make_iteration()
    
    def current_eps(self):
        return Vector(self.current - self.previous).norm()

    def make_iteration(self):
        self.iteration += 1
        error = Vector(self.A * self.current) - self.f
        self.previous, self.current = self.current, Vector(error * (-self.tau) + self.current)
        if self.verbose:
            print("iteration {}, solution:\n{}".format(self.iteration, self.current))

In [316]:
A = np.array([[0, 1], [2, 3]])
b = np.array([[1], [2]])

In [317]:
task = PMTask(Matrix(A), Vector(b), verbose=True)

Gershgorin tau approximation: 0.14285714285714285 (lambd_min=-8.0, lambd_max=22.0)


In [318]:
task.A

[4.0, 6.0]
[6.0, 10.0]

In [319]:
task.f

[4.0]
[7.0]

In [320]:
task.current_eps()

14142134.209517388

In [321]:
task.run()

iteration 1, solution:
[0.1428571428571429]
[-0.2857142857142856]

iteration 2, solution:
[0.8775510204081632]
[0.9999999999999998]

iteration 3, solution:
[0.09037900874635607]
[-0.18075801749271125]

iteration 4, solution:
[0.765097875885048]
[0.9999999999999996]

iteration 5, solution:
[0.04218480395073543]
[-0.08436960790146952]

iteration 6, solution:
[0.6618245798944319]
[0.9999999999999993]

iteration 7, solution:
[-0.002075180045242875]
[0.004150360090487304]

iteration 8, solution:
[0.5669817570459068]
[0.9999999999999993]

iteration 9, solution:
[-0.042722104123182336]
[0.085444208246366]

iteration 10, solution:
[0.4798812054503224]
[0.9999999999999994]

iteration 11, solution:
[-0.08005091194986125]
[0.16010182389972372]

iteration 12, solution:
[0.399890902964582]
[0.9999999999999994]

iteration 13, solution:
[-0.11433247015803577]
[0.22866494031607287]

iteration 14, solution:
[0.3264304210899222]
[0.9999999999999994]

iteration 15, solution:
[-0.1458155338186043]
[0.2916